In [14]:
from fastai.vision.widgets import *
from fastai.vision.widgets import *
from fastai.vision.all import * 
import fastbook
fastbook.setup_book()
from fastai.imports import *

### Joe's EPIC Bear Classifier


Are you stranded in the middle of the woods with a bear like figure is charging at you at full speed and you don't know what you're up against? Well lucky for you, with this bear classfier, you'll know EXACTLY what force of nature you are up against!


Disclaimer: This classifier only classifies grizzly, black and teddy bears. 

In [15]:
# MAKE SURE TO UPLOAD UTILS FILE EACH TIME (IT'S IN E:\)
# utils has a bunch of functions that are used throughout the course

# key come from MS Azure bing search api
# this api allows us to download images from the internet, based on given term
key = os.environ.get('AZURE_SEARCH_KEY', 'b9977b83e677493ead5e6258eb65fd8d')


In [16]:
# Now let's create a model that can differentiate between different bear types
bear_types = 'grizzly', 'black', 'teddy'  
path = Path('bears')

In [17]:
# if the path "bears" does not exist, create a new one and download images for training 
# otherwise, use folder that is already there 

if not path.exists():
    path.mkdir()

    # for each of the bear types, search bing for that term
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)

        # use bear type + bear
        # create separate directories for each type (150 of each)
        results = search_images_bing(key, f'{o} bear')

        # fast ai function
        download_images(dest, urls=results.attrgot('contentUrl'))

In [18]:
# show all images downloaded 

fns = get_image_files(path)
fns

# check if every file downloaded is an image file
failed = verify_images(fns)
failed

# delete all failed files/non image files
failed.map(Path.unlink)


(#0) []

In [19]:
class DataLoaders(GetAttr):
    def __init__(self, *loaders): self.loaders = loaders
    train,valid = add_props(lambda i,self: self[i])

In [20]:
# Create DataBlocks
# ALWAYS RUN THIS INITIAL DATABLOCK FIRST 
bears = DataBlock(
    
    # Image --> input data
    # Category --> labels
    blocks = (ImageBlock, CategoryBlock),

    # get list of files with get_image_files
    get_items=get_image_files,

    # random set for validation uses 30% of data
    splitter = RandomSplitter(valid_pct = 0.3, seed=42),

    # parent_label --> labels each item based on the parent label 
    # e.g. grizzly, black etc 
    get_y = parent_label,       # side note: commas at the end are important

    # tfms = transform
    # Resize(128) --> resize to 128x128 pixel image, having a square image is better (for simplicity)
    # Default is to grab the centre of each image
    # This cropping approach "cuts" information
    
    item_tfms = Resize(128)

)

# Data loader stiches a bunch of images together (usually 64) and sends them 
# to the GPU at once (allows for speed). 

In [21]:
# Now call data loaders
# dls --> variable for our dataloaders
dls = bears.dataloaders(path) 

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


In [22]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5), 
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


In [23]:
# From our model from lesson 1

# This is the actual syntax to run the training model

learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

# this "learn" variable is our model 

epoch,train_loss,valid_loss,error_rate,time
0,1.839638,0.214726,0.090090,01:11


G:\LiterallyAnythingElse\anaconda\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


epoch,train_loss,valid_loss,error_rate,time
0,0.218249,0.096434,0.036036,01:21
1,0.147106,0.072471,0.027027,01:12
2,0.104118,0.070658,0.027027,01:11
3,0.087651,0.063354,0.027027,01:14


In [25]:
# with our created model "learn", we can now copy this to a server etc
# treat it as a predefined program
learn.export()
path = Path()
path.ls(file_exts = '.pkl')

# our export is called 'export.pkl' be default


(#1) [Path('export.pkl')]

In [27]:
# Assign path and widgets
path = Path()

learn_inf = load_learner(path/'export.pkl', cpu=True) 
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [28]:
# We now create an event handler which executes a function based on what widget is activated
# In this case, when classify button is pressed....

def on_click(change):
    # set img as most recent image
    img = PILImage.create(btn_upload.data[-1])
    
    # clear output widget
    out_pl.clear_output()
    
    # display img in output widget
    with out_pl: display(img.to_thumb(128,128))
     
    # use model to predict what uploaded image is 
    pred,pred_idx,probs = learn_inf.predict(img)
    
    # Replace our label with prediction
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
   



In [29]:
# Now we create a vertical box (VBox) to complete our GUI
btn_upload.observe(on_click, names = ['data'])

In [30]:
display(VBox([widgets.Label('Select your bear!'), btn_upload, out_pl, lbl_pred]))
# add upload button, event handler button, output window and label

# It's the exact same buttons, but now we organised them in a "prototpye" application

In [31]:
# install voila
!pip install voila
!jupyter serverextension enable --sys-prefix voila

Enabling: voila
- Writing config: G:\LiterallyAnythingElse\anaconda\etc\jupyter
    - Validating...
      voila 0.2.10 ok
